<a href="https://colab.research.google.com/github/Ropicat/Data-71200-fall-2025/blob/main/Splitting_of_the_Data_TRAINING_Nov_25_update_of_DATA_71200_PROJECT_ONE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Task
# STEP ONE TO STEP SIX
#Download "Squirrel Census Data
#----------------------------------------
Of the hundreds of thousands of datasets maintained by NYC, Squirrel Census Data is one of the more whimsical. Given the various variables, many of which appear to have consistency issues, it is unclear whether the data collection was intended to yield any valuable information. Several studies have been conducted, the latest from Carnegie Mellon, but the results have been ambiguous at best.

I plan to extensively use dummy variables to determine whether the physical and behavioral data can predict the individual squirrel's behavior toward humans (running away or being indifferent or coming toward you). So this will be a Multinomial Logistic Regression, with  Again, it is not clear if there are any clear analytical outcomes, but seeking insights into Squirrel behavior is a fun exercise!

Also please note that initially I did not undertake splitting the data into training and testing before cleaning and tranforming dthe data, because I did not think none of my data cleanup processes and data transformation processes risked data leakage. Still, I was aware of the broad instruction of the Project, and I understand I should have articulated the reason for my not splitting the data. My fault.

What you see is the Project 1 which fully follows the instruction



SECTION 1: "Step One" Find and Download a Dataset

SECTION 2: "Step Two" Divide into a training set and a testing set

SECTION 3: "Step Three" Data Cleanup

SECTION 4: "Step Four" Data Transformation

SECTION 5: "Step Five" Explore the training set

SECTION 6: "Step Six" Visualize the data in my training set



# Data extracted from the share link, the original file is located in my Google Drive

In [ ]:
#from google.colab import drive
import requests
import gdown

# Construct the download URL
download_url = 'https://drive.google.com/uc?export=download&id=1TmUCaPBaOSj48J_D2q9odCiT-3T3SD6S'

# Define the local filename
local_filename = '2018_Central_Park_Squirrel_Census_-_Squirrel_Data_29250910.csv'

# Download the file
response = requests.get(download_url, stream=True)
response.raise_for_status()  # Raise an exception for bad status codes

with open(local_filename, 'wb') as f:
    for chunk in response.iter_content(chunk_size=8192):
        f.write(chunk)

print(f"Downloaded '{local_filename}' successfully.")

Downloaded '2018_Central_Park_Squirrel_Census_-_Squirrel_Data_29250910.csv' successfully.


In [ ]:
import pandas as pd

# Load the downloaded CSV file into a pandas DataFrame
df = pd.read_csv('2018_Central_Park_Squirrel_Census_-_Squirrel_Data_29250910.csv')

# Display the first few rows of the DataFrame
display(df.head())

,X,Y,Unique Squirrel ID,Hectare,Shift,Date,Hectare Squirrel Number,Age,Primary Fur Color,Highlight Fur Color,...,Kuks,Quaas,Moans,Tail flags,Tail twitches,Approaches,Indifferent,Runs from,Other Interactions,Lat/Long
0,-73.956134,40.794082,37F-PM-1014-03,37F,PM,10142018,3,NaN,NaN,NaN,...,False,False,False,False,False,False,False,False,NaN,POINT (-73.9561344937861 40.7940823884086)
1,-73.968857,40.783783,21B-AM-1019-04,21B,AM,10192018,4,NaN,NaN,NaN,...,False,False,False,False,False,False,False,False,NaN,POINT (-73.9688574691102 40.7837825208444)
2,-73.974281,40.775534,11B-PM-1014-08,11B,PM,10142018,8,NaN,Gray,NaN,...,False,False,False,False,False,False,False,False,NaN,POINT (-73.97428114848522 40.775533619083)
3,-73.959641,40.790313,32E-PM-1017-14,32E,PM,10172018,14,Adult,Gray,NaN,...,False,False,False,False,False,False,False,True,NaN,POINT (-73.9596413903948 40.7903128889029)
4,-73.970268,40.776213,13E-AM-1017-05,13E,AM,10172018,5,Adult,Gray,Cinnamon,...,False,False,False,False,False,False,False,False,NaN,POINT (-73.9702676472613 40.7762126854894)


In [ ]:
# Count the number of rows and columns
num_rows, num_cols = df.shape
print(f"Number of rows: {num_rows}")
print(f"Number of columns: {num_cols}")

# Count the number of NaN values for each column
nan_counts = df.isnull().sum()

# Display the NaN counts
print("\nNaN counts per column:")
display(nan_counts)

Number of rows: 3023
Number of columns: 31

NaN counts per column:


,0
X,0
Y,0
Unique Squirrel ID,0
Hectare,0
Shift,0
Date,0
Hectare Squirrel Number,0
Age,121
Primary Fur Color,55
Highlight Fur Color,1086


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Define the target column for stratification.
# Based on the task description "predict the individual squirrel's behavior toward humans
# (running away or being indifferent or coming toward you)", 'Runs from' is chosen as a
# representative target for stratification. Other columns like 'Indifferent' or 'Approaches'
# could also be used or a combined target variable could be created.
target_column = 'Runs from'

# Ensure the target column has no missing values, as train_test_split cannot stratify on NaNs.
# From the `nan_counts` in the notebook state, 'Runs from' has 0 NaNs, so no explicit
# dropping of rows based on this column is needed at this stage.

# Split the DataFrame into training and testing sets.
# The `stratify` parameter ensures that the proportion of values in `target_column`
# is approximately the same in both the training and testing sets.
# `test_size=0.2` is a common split ratio, using 20% of the data for testing.
# `random_state=42` ensures reproducibility of the split.
train_df, test_df = train_test_split(df, test_size=0.2, stratify=df[target_column], random_state=42)

# Assign the training and testing DataFrames to X_train and X_test, respectively.
# This fulfills the requirement for subsequent cells that use X_train (e.g., display(X_train.head())
# and X_train.to_csv()). Although the original instruction said "No need to assign X_train, y_train,
# X_test, y-test", the subsequent use of X_train implies it must be defined.
X_train = train_df
X_test = test_df

print(f"Original DataFrame shape: {df.shape}")
print(f"Training set shape (X_train): {X_train.shape}")
print(f"Testing set shape (X_test): {X_test.shape}")

Original DataFrame shape: (3023, 31)
Training set shape (X_train): (2418, 31)
Testing set shape (X_test): (605, 31)


In [ ]:
print(df.columns.tolist())

['X', 'Y', 'Unique Squirrel ID', 'Hectare', 'Shift', 'Date', 'Hectare Squirrel Number', 'Age', 'Primary Fur Color', 'Highlight Fur Color', 'Combination of Primary and Highlight Color', 'Color notes', 'Location', 'Above Ground Sighter Measurement', 'Specific Location', 'Running', 'Chasing', 'Climbing', 'Eating', 'Foraging', 'Other Activities', 'Kuks', 'Quaas', 'Moans', 'Tail flags', 'Tail twitches', 'Approaches', 'Indifferent', 'Runs from', 'Other Interactions', 'Lat/Long']


In [ ]:
import pandas as pd
from google.colab import drive

# Mount Google Drive if not already mounted (though it should be from previous steps)
try:
    drive.mount('/content/drive', force_remount=True)
except:
    print('Google Drive already mounted.')

# Define the full path where you want to save the X_train.csv file in your Google Drive
save_path = https://drive.google.com/file/d/1CUUG9fN5xSFNuJWkTuWeSkjllf5mytqE/view?usp=sharing

# Save the X_train DataFrame to the specified path as a CSV file
X_train.to_csv(save_path, index=False)

print(f"X_train DataFrame successfully saved to {save_path}")

Mounted at /content/drive
X_train DataFrame successfully saved to /content/drive/My Drive/squarrel_train_data.csv


In [ ]:
import pandas as pd
from google.colab import drive

# Mount Google Drive if not already mounted
try:
    drive.mount('/content/drive', force_remount=True)
except:
    print('Google Drive already mounted.')

# Define the full path where you want to save the X_test.csv file in your Google Drive
save_path_test = '/content/drive/My Drive/squarrel_test_data.csv'

# Save the X_test DataFrame to the specified path as a CSV file
X_test.to_csv(save_path_test, index=False)

print(f"X_test DataFrame successfully saved to {save_path_test}")

Mounted at /content/drive
X_test DataFrame successfully saved to /content/drive/My Drive/squarrel_test_data.csv


# Task
Mount Google Drive and save the `X_train` DataFrame as a CSV file named `X_train.csv` to your Google Drive. Then, confirm that `X_train.csv` has been successfully saved to Google Drive.

## Mount Google Drive and Save X_train

### Subtask:
Mount Google Drive and then save the `X_train` DataFrame to a CSV file named `X_train.csv` in your Google Drive.


**Reasoning**:
Mount Google Drive and save the `X_train` DataFrame to a CSV file in Google Drive as instructed by the subtask.



In [ ]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Define the full path where you want to save the X_train.csv file in your Google Drive
save_path = '/content/drive/My Drive/X_train.csv'

# Save the X_train DataFrame to the specified path as a CSV file
X_train.to_csv(save_path, index=False)

print(f"X_train DataFrame successfully saved to {save_path}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
X_train DataFrame successfully saved to /content/drive/My Drive/X_train.csv


## Final Task

### Subtask:
Confirm that the `X_train.csv` file has been successfully saved to Google Drive.
